In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import os
import h5py
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import tensorflow as tf

In [82]:
import numpy as np
import h5py
import glob # Used to find all file paths
import os

BASE_PATH = '/content/drive/MyDrive/DeepLearning Project/Final Project data/Cross'

TRAIN_DIR = os.path.join(BASE_PATH, 'train/')
TEST1_DIR = os.path.join(BASE_PATH, 'test1/')
TEST2_DIR = os.path.join(BASE_PATH, 'test2/')
TEST3_DIR = os.path.join(BASE_PATH, 'test3/')

# Each file has 248 sensor readings (rows) and 35624 time steps (columns)
N_CHANNELS = 248
N_TIMESTEPS = 35624

# The 4 states we want to classify
TASKS = ['rest', 'task_motor', 'task_story_math', 'task_working_memory']
# Map tasks to integer labels
task_to_label = {task: i for i, task in enumerate(TASKS)}

# Function to load data from a list of file paths
def load_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        # Extractin the label
        filename = file_path.split('/')[-1]

        #handling the different task naming conventions
        if 'rest' in filename:
            labels.append(task_to_label['rest'])
        elif 'motor' in filename:
            labels.append(task_to_label['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(task_to_label['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(task_to_label['task_working_memory'])
        else:
            # iff a file doesn't match
            print(f"Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            # Instead of guessing the dataset name, we get the first key from the file
            # This is robust because we know there is only one dataset per file[cite: 10].
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    #convert to numpy arrays
    return np.array(data), np.array(labels)

train_files = glob.glob(f"{TRAIN_DIR}/*.h5")
test1_files = glob.glob(f"{TEST1_DIR}/*.h5")
test2_files = glob.glob(f"{TEST2_DIR}/*.h5")
test3_files = glob.glob(f"{TEST3_DIR}/*.h5")

X_train, y_train = load_data(train_files)
X_test1, y_test1 = load_data(test1_files)
X_test2, y_test2 = load_data(test2_files)
X_test3, y_test3 = load_data(test3_files)


print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test1: {X_test1.shape}")
print(f"Unique labels: {np.unique(y_train)}")
print(f"Number of training samples: {len(X_train)}")

Loading training data...
Loading test set 1...
Loading test set 2...
Loading test set 3...

Data loading complete.
Shape of X_train: (64, 248, 35624)
Shape of y_train: (64,)
Shape of X_test1: (16, 248, 35624)
Unique labels: [0 1 2 3]
Number of training samples: 64


# Preprocessing

In [83]:
from sklearn.preprocessing import StandardScaler

# --- Downsampling ---
# The original sample rate is 2034Hz
#take every 10th sample
DOWNSAMPLE_FACTOR = 10
X_train_ds = X_train[:, :, ::DOWNSAMPLE_FACTOR]
X_test1_ds = X_test1[:, :, ::DOWNSAMPLE_FACTOR]
X_test2_ds = X_test2[:, :, ::DOWNSAMPLE_FACTOR]
X_test3_ds = X_test3[:, :, ::DOWNSAMPLE_FACTOR]

N_TIMESTEPS_DS = X_train_ds.shape[2]
print(f"Original number of time steps: {N_TIMESTEPS}")
print(f"Downsampled number of time steps: {N_TIMESTEPS_DS}")


# --- Time-wise Normalization ---
#normalize each channel's time-series independently.
def normalize_data(data):
    # Data shape is (n_samples, n_channels, n_timesteps)
    # We want to scale each of the (n_samples * n_channels) time series

    # Reshape to (n_samples * n_channels, n_timesteps) to apply StandardScaler
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

print("\nNormalizing data...")
X_train_norm = normalize_data(X_train_ds)
X_test1_norm = normalize_data(X_test1_ds)
X_test2_norm = normalize_data(X_test2_ds)
X_test3_norm = normalize_data(X_test3_ds)

#DL models in Keras often expect the channel dimension last
#reshaping from (samples, channels, timesteps) to (samples, timesteps, channels)
X_train_final = np.transpose(X_train_norm, (0, 2, 1))
X_test1_final = np.transpose(X_test1_norm, (0, 2, 1))
X_test2_final = np.transpose(X_test2_norm, (0, 2, 1))
X_test3_final = np.transpose(X_test3_norm, (0, 2, 1))

print("Normalization complete.")
print(f"Final shape of training data for the model: {X_train_final.shape}")

Original number of time steps: 35624
Downsampled number of time steps: 3563

Normalizing data...
Normalization complete.
Final shape of training data for the model: (64, 3563, 248)


In [84]:
# Code Block 3: 1D CNN Model Architecture
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        #1st convolutional block
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        #2nd convolutional block
        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # 3rd convolutional block
        Conv1D(filters=256, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # Flatten the features and feed to dense layers
        Flatten(),

        # dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

# define model parameters
INPUT_SHAPE = (N_TIMESTEPS_DS, N_CHANNELS)
NUM_CLASSES = len(TASKS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 3563, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3563, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 890, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 890, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 890, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 222, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 222, 256)       │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 222, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 55, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 14080)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 128)            │     1,802,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,373,444 (9.05 MB)

 Trainable params: 2,372,548 (9.05 MB)

 Non-trainable params: 896 (3.50 KB)

In [85]:
#CNN Training and Evaluation
history_cnn = cnn_model.fit(
    X_train_final,
    y_train,
    epochs=20,          # You can adjust the number of epochs
    batch_size=16,      # Smaller batch size for better generalization
    validation_split=0.2 # Use 20% of training data for validation
)

# Evaluate on test Sets
loss1_cnn, acc1_cnn = cnn_model.evaluate(X_test1_final, y_test1, verbose=0)
print(f"accuracy on test set 1: {acc1_cnn * 100:.2f}%")
loss2_cnn, acc2_cnn = cnn_model.evaluate(X_test2_final, y_test2, verbose=0)
print(f"accuracy on test set 2: {acc2_cnn * 100:.2f}%")
loss3_cnn, acc3_cnn = cnn_model.evaluate(X_test3_final, y_test3, verbose=0)
print(f"accuracy on test set 3: {acc3_cnn * 100:.2f}%")

Starting CNN model training...
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.4034 - loss: 2.3540 - val_accuracy: 0.2308 - val_loss: 5.3566
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7961 - loss: 1.5879 - val_accuracy: 0.6154 - val_loss: 3.8721
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8988 - loss: 0.7588 - val_accuracy: 0.6154 - val_loss: 5.6472
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8357 - loss: 1.3855 - val_accuracy: 0.5385 - val_loss: 11.9915
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7768 - loss: 2.2987 - val_accuracy: 0.6923 - val_loss: 2.1674
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9760 - loss: 0.8614 - val_accuracy: 0.6923 - val_loss: 2.3752
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9420 - loss: 1.3984 - val_accuracy: 0.9231 - val_loss: 0.0657
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9561 - loss: 0.8939 - val_accuracy: 0.6154 - v

In [91]:
#Hybrid CNN-LSTM Model Architecture
from tensorflow.keras.layers import LSTM

def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # Convolutional layers to extract features
        Conv1D(filters=64, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        Conv1D(filters=128, kernel_size=10, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=4),

        # LSTM layer to model temporal sequences of the extracted features
        LSTM(128, return_sequences=False), # return_sequences=False  it's the last recurrent layer

        # Dense layers for classification
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_13 (Conv1D)              │ (None, 3563, 64)       │       158,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 3563, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_13 (MaxPooling1D) │ (None, 890, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 890, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 890, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 222, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 390,212 (1.49 MB)

 Trainable params: 389,828 (1.49 MB)

 Non-trainable params: 384 (1.50 KB)

In [87]:
#CNN-LSTM Training and Evaluation
print("Starting CNN-LSTM model training...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2
)

# Evaluate on test sets
loss1_hybrid, acc1_hybrid = cnn_lstm_model.evaluate(X_test1_final, y_test1, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 1: {acc1_hybrid * 100:.2f}%")
loss2_hybrid, acc2_hybrid = cnn_lstm_model.evaluate(X_test2_final, y_test2, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 2: {acc2_hybrid * 100:.2f}%")
loss3_hybrid, acc3_hybrid = cnn_lstm_model.evaluate(X_test3_final, y_test3, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 3: {acc3_hybrid * 100:.2f}%")

Starting CNN-LSTM model training...
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.3444 - loss: 1.3183 - val_accuracy: 0.6154 - val_loss: 1.1104
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8227 - loss: 0.7680 - val_accuracy: 0.5385 - val_loss: 0.9905
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7799 - loss: 0.6006 - val_accuracy: 0.6154 - val_loss: 0.8885
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8498 - loss: 0.4025 - val_accuracy: 0.6154 - val_loss: 0.8522
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9556 - loss: 0.2351 - val_accuracy: 0.6154 - val_loss: 0.7353
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 0.1535 - val_accuracy: 0.6923 - val_loss: 0.6267
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9922 - loss: 0.1610 - val_accuracy: 0.6923 - val_loss: 0.6255
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 1.0000 - loss: 0.0948 - val_accuracy: 0.6923

In [89]:
#EEGNet model
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Dropout, BatchNormalization, Activation, AveragePooling2D, Flatten, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.callbacks import EarlyStopping

def build_eegnet_model(num_classes, channels, timesteps, dropout_rate=0.5):
    input_layer = Input(shape=(channels, timesteps, 1))

    # temporal convolution
    block1 = Conv2D(16, (1, 64), padding='same', use_bias=False)(input_layer)
    block1 = BatchNormalization()(block1)

    #Depthwise spatial convolution
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=2, depthwise_constraint=tf.keras.constraints.max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropout_rate)(block1)

    # separable convolution
    block2 = SeparableConv2D(32, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = Dropout(dropout_rate)(block2)

    # classification head
    flatten_layer = Flatten()(block2)
    dense_layer = Dense(num_classes, kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten_layer)
    output_layer = Activation('softmax')(dense_layer)

    return Model(inputs=input_layer, outputs=output_layer)

# Reshape data
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test1_eegnet = X_test1_norm[..., np.newaxis]
X_test2_eegnet = X_test2_norm[..., np.newaxis]
X_test3_eegnet = X_test3_norm[..., np.newaxis]

print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=N_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

print("\nStarting EEGNet model training with Early Stopping...")
history_eegnet = eegnet_model.fit(
    X_train_eegnet,
    y_train,
    epochs=50,  # We can still set a high number, but early stopping will likely stop it sooner
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

print("\nEvaluating EEGNet model on test sets")
loss1_eegnet, acc1_eegnet = eegnet_model.evaluate(X_test1_eegnet, y_test1, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 1: {acc1_eegnet * 100:.2f}%")

loss2_eegnet, acc2_eegnet = eegnet_model.evaluate(X_test2_eegnet, y_test2, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 2: {acc2_eegnet * 100:.2f}%")

loss3_eegnet, acc3_eegnet = eegnet_model.evaluate(X_test3_eegnet, y_test3, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 3: {acc3_eegnet * 100:.2f}%")

Shape of data for EEGNet: (64, 248, 3563, 1)


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 248, 3563, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 248, 3563, 16)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 248, 3563, 16)  │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 3563, 32)    │         7,936 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 1, 3563, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 3563, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 1, 890, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 1, 890, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 1, 890, 32)     │         1,536 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1, 890, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1, 890, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 1, 111, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 1, 111, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 3552)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 4)              │        14,212 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,028 (97.77 KB)

 Trainable params: 24,868 (97.14 KB)

 Non-trainable params: 160 (640.00 B)


Starting EEGNet model training with Early Stopping...
Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 76s 17s/step - accuracy: 0.5311 - loss: 1.5636 - val_accuracy: 0.3846 - val_loss: 1.3184
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 73s 17s/step - accuracy: 0.7496 - loss: 0.5258 - val_accuracy: 0.6923 - val_loss: 1.2582
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 72s 17s/step - accuracy: 0.9249 - loss: 0.3209 - val_accuracy: 0.7692 - val_loss: 1.1987
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 72s 17s/step - accuracy: 0.9457 - loss: 0.2629 - val_accuracy: 0.8462 - val_loss: 1.1376
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 74s 17s/step - accuracy: 0.9614 - loss: 0.1623 - val_accuracy: 0.7692 - val_loss: 1.0735
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 73s 17s/step - accuracy: 0.9739 - loss: 0.1462 - val_accuracy: 0.7692 - val_loss: 1.0376
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 73s 17s/step - accuracy: 0.8957 - loss: 0.1616 - val_accuracy: 0.9231 - val_loss: 1.0201
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 73s 17s/step - accuracy: 0.9692 - 